## Battery Aging Challenge
Estimating the remaining capacity form field data.

### Input
* 6 months of battery voltage, battery current and battery temperature (stored in monthly .csv files: 2020-04 to 2020-10)
* regular testbench capacity measurements for the first 3 months
* chemistry: NMC

### Task
Estimate the remaining capacity at the following dates:
* 2020-07-11
* 2020-08-04
* 2020-08-28
* 2020-09-22
* 2020-10-01

### Gotcha
Capacity test related time series were removed from the data to prevent the solution of plain coulomb counting for those tests. Therefore you will recognize data gaps at certain dates.

In [4]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from utils import plot_battery_data, plot_testbench_results
import numpy as np
%matplotlib widget  
from ipywidgets import *
import matplotlib.pyplot as plt


In [5]:
# load dataset
battery_data = pd.concat([pd.read_csv('data/battery_data/2020-{}.csv'.format(month)) for month in range(4, 10)])
battery_data['datetime'] = pd.to_datetime(battery_data['timestamp'], unit='s')
battery_data['power'] = battery_data['current'] * battery_data['voltage']
# extract and plot one month.
subset = battery_data[battery_data['timestamp']<datetime.datetime(2020, 4, 29, 0, 0, 0).timestamp()]
plt.figure()
# change the key to look for motifs in different datatypes
key = 'voltage'
plt.plot(subset['datetime'], subset[key])
plt.show()
# Use this graph to identify the motif you want to search for. Enter the start and end date and time for your motif in the next cell. 
# Or to use default motifs skip this

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Enter the start and end date and time for your motif here. Note that for some reason you need to add one hour on
# here are some premade motifs

# voltage hour
# motif = battery_data[(datetime.datetime(2020, 4, 2, 12, 30, 47).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 2, 13, 33, 57).timestamp() > battery_data['timestamp'])]

# voltage charge
# so here we are just searching for the charge part of the cycle
motif = battery_data[(datetime.datetime(2020, 4, 2, 15, 26, 50).timestamp() < battery_data['timestamp']) &
                     (datetime.datetime(2020, 4, 2, 15, 34, 38).timestamp() > battery_data['timestamp'])]

# power charging
# motif = battery_data[(datetime.datetime(2020, 4, 2, 13, 29, 38).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 2, 13, 33, 56).timestamp() > battery_data['timestamp'])]

# current hour
# motif = battery_data[(datetime.datetime(2020, 4, 2, 12, 30, 53).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 2, 13, 34, 8).timestamp() > battery_data['timestamp'])]

# day
# motif = battery_data[(datetime.datetime(2020, 4, 2, 11, 34, 3).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 3, 5, 1, 17).timestamp() > battery_data['timestamp'])]


# The data is very high resolution, so it can be slow to search. Best to reduce resolution with sampling
# normalise the motif to mean of zero for comparison
sampling = 5
# plot the motif
plt.figure()
data, motif = battery_data[::sampling][key].to_numpy(), motif[::sampling][key].to_numpy()
plt.plot(motif)

# This is the motif we will search for

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:

# Here we perform a very simple (and slow!) search for the motif, where 'errors' is difference between the motif and the sample
motif_norm = motif - np.mean(motif)

l = len(motif)
errors = []
for start in range(data.shape[0] - l):
    samp = data[start:start + l] - np.mean(data[start:start + l])
    errors.append(np.mean((samp - motif_norm)**2))

In [8]:
plt.figure()
plt.plot(errors)
plt.show()
# Zoom in very far to see the errors near to zero - these are the parts of the data which fit the motif

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
error_cutoff = 0.0001
dataset_v = []
dataset_c = []
prev_start = - l
n = 0
fig, axs = plt.subplots(3)
fig.set_size_inches(12, 10)
for start, error in enumerate(errors):
    if (error < error_cutoff) & (start - l >= prev_start):
        prev_start = start
        c = start/len(errors)
        # plt.plot(data[start:start + l] - np.mean(data[start:start + l]) - motif, color = (c, 1-c, 0), alpha=0.02)
        for ax, key in zip(axs.ravel(), ['current', 'voltage', 'power']):
            data = battery_data[key][::sampling].to_numpy()
            ax.plot(data[start:start + l], color = (c, 1-c, 1-c), alpha=0.01)
            ax.set_ylabel(key)
            if key=='current':
                dataset_c.append(data[start:start + l])
            if key=='voltage':
                dataset_v.append(data[start:start + l])
        # plt.plot(, color = (1, 1, 0), alpha=1)
        n+=1
print(n)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2672


In [51]:
print(np.array(dataset).shape)
line_v = np.mean(dataset_v)
lines_v = np.mean(dataset_v, axis=1)
plt.figure()
fig, axs = plt.subplots(3)
# plt.plot(lines)
# linear_model=np.polyfit(np.arange(len(lines)),lines,1)
# linear_model_fn=np.poly1d(linear_model)
# x_s=np.arange(len(lines))
plot_line_v = []
sum = 0
for i in range(len(lines_v)):
    sum += lines_v[i]
    plot_line_v.append(sum/(i+1))
voltage_line = np.array(plot_line_v)[250:] 
axs[0].plot(voltage_line,color="red")
axs[0].title.set_text('voltage')
# jump=80
# for i in range(0,len(lines_v),jump):
#     if i+jump<len(lines_v):
#         cur_lines = 0
#         for j in range(jump):
#             cur_lines += lines_v[i+j]
#         plot_line_v.append(cur_lines/jump)
# voltage_line = np.array(plot_line_v)-line_v + 1
# plt.plot(voltage_line, color="red")
line_c = np.mean(dataset_c)
lines_c = np.mean(dataset_c, axis=1)
# plt.figure()
# plt.plot(lines)
# linear_model=np.polyfit(np.arange(len(lines)),lines,1)
# linear_model_fn=np.poly1d(linear_model)
# x_s=np.arange(len(lines))
plot_line_c = []
sum = 0
for i in range(len(lines_c)):
    sum += lines_c[i]
    plot_line_c.append(sum/(i+1))
current_line = np.array(plot_line_c)[250:]
axs[1].plot(current_line,color="Green")
axs[1].title.set_text('current')
# jump=250
# for i in range(0,len(lines_c),jump):
#     if i+jump<len(lines_c):
#         cur_lines = 0
#         for j in range(jump):
#             cur_lines += lines_c[i+j]
#         plot_line_c.append(cur_lines/jump)
# current_line = np.array(plot_line_c)-line_c + 
# plt.plot(current_line, color="Green")
power_line = np.multiply(voltage_line, current_line)
axs[2].plot(power_line, color="Blue")
axs[2].title.set_text('power')
plt.show()

(2672, 94)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:

plt.figure()
plt.imshow(np.array(dataset - np.mean(dataset, axis=0)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure()
step = 100
for s in range(0, n-step, step):
    plt.plot(abs(np.mean(dataset[s:s+step] - np.mean(dataset, axis=0), axis=0)), c =( s/887, 1-s/887, 1-s/887), linewidth=0.7)
    # plt.plot(np.mean(dataset[s:s+step] - np.mean(dataset, axis=0), axis=0), c =( s/887, 1-s/887, 1-s/887), linewidth=0.7)
